# Inferência em Lógica de Primeira Ordem

`Expr`: tipo de dados para sentenças lógicas; mecanismo para representar expressões<br>
`FolKB`: base de conhecimento para lógica de primeira ordem.<br>
Forward chaining e backward chaining para `FolKB` e seu uso na base `crime_kb`.

In [16]:
from utils import *
from logic import *

## Sentenças lógicas

The `Expr` class is designed to represent any kind of mathematical expression. The simplest type of `Expr` is a symbol, which can be defined with the function `Symbol`:

In [17]:
Symbol('x')

x

In [18]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')
P & ~Q

(P & ~Q)

In [19]:
Pxy = P(x, y)
Pxy.op

'P'

In [20]:
Pxy.args

(x, y)

## Operadores para construção de sentenças lógicas

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

In [21]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

In [22]:
expr('~(P & Q)  ==>  (~P | ~Q)') #expr pega uma string e faz o parse parauma instância de Expr

(~(P & Q) ==> (~P | ~Q))

In [23]:
expr('sqrt(b ** 2 - 4 * a * c)')

sqrt(((b ** 2) - ((4 * a) * c)))

## First-Order Logic Knowledge Bases: `FolKB`

### Exemplo: Criminal KB

<em>The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American.</em><br/>

Primeiro passo: extrair os fatos e converte-los isoladamente para cláusulas FOL. 

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Segundo passo: unir os fatos em cláusulas compostas.

* `American(x)` $\land$ `Weapon(y)` $\land$ `Sells(x, y, z)` $\land$ `Hostile(z)` $\implies$ `Criminal(x)`

In [24]:
clauses = []

In [25]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"The country Nono, an enemy of America"</em><br/>

In [26]:
clauses.append(expr("Enemy(Nono, America)"))

<em>"Nono ... has some missiles"</em><br/>

In [27]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>"All of its missiles were sold to it by Colonel West"</em><br/>

In [28]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, who is American"</em><br/>

In [29]:
clauses.append(expr("American(West)"))

É sabido ainda que mísseis são armas e inimigos são hostis.

In [30]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

### KB criada

In [31]:
crime_kb = FolKB(clauses)

## Inferência em FOL

Os algoritmos **Forward chaining** e **Backward chaining** utilizam um componente/processo-chave chamado <strong>Unificação</strong>: a substituição de variáveis por constantes é um exemplo típico.<br>
<br>`unify`: algoritmo recursivo (dicionário).

In [32]:
unify(expr('x'), 3)

{x: 3}

In [33]:
unify(expr('A(x)'), expr('A(B)'))

{x: B}

In [34]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

{x: Bella, y: Dobby}

In [35]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

None
None


### Forward Chaining 

Tentativa de unificar cada uma das premissas com a cláusula na `KB`. Se possível, a conclusão é adicionada à `KB`. Processo de inferência é repetido até que a query possa ser respondida ou até que nenhuma sentença possa ser adicionada.

A função `fol_fc_ask` é um gerador que leva a todas as substituições que validam a query.

In [36]:
# todas as nações hostis da base
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


#### Adicionando novos dados à base: 

função `tell`

In [37]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


<strong><em>Note</em>:</strong> `fol_fc_ask` faz mudanças na `KB`.

### Backward Chaining 

Parte do objetivo/query, utilizando as regras existentes na base para encontrar os fatos. 

`fol_bc_or` e `fol_bc_and`

O método `ask` de `FolKB` usa `fol_bc_ask` e dispara a primeira substituição retornada pelo gerador para responder à query.

In [38]:
crime_kb = FolKB(clauses)
crime_kb.ask(expr('Hostile(x)'))

{v_5: x, x: Nono}

In [39]:
crime_kb.ask(expr('Hostile(Brasil)'))

False

### Exemplo: Coloração de mapas

<img src="colormapa.png" width="604" height="250" border="1" alt="Coloração de mapas">

In [40]:
import sys
#sys.setrecursionlimit(15000)
clauses = []
clauses.append(expr("Adj(R,G)"))
clauses.append(expr("Adj(G,R)"))
clauses.append(expr("Adj(R,B)"))
clauses.append(expr("Adj(B,R)"))
clauses.append(expr("Adj(B,G)"))
clauses.append(expr("Adj(G,B)"))
clauses.append(expr("Adj(R)"))
clauses.append(expr("Adj(B)"))
clauses.append(expr("Adj(G)"))
clauses.append(expr("(Adj(ao,tn) & Adj(ao,am) & Adj(tn,am) & Adj(tn,q) & Adj(am,q) & Adj(am,ngs) & Adj(q, ngs) & Adj(am,v) & Adj(ngs,v) & Adj(t)) ==> Colorido(ao,tn,q,am,ngs,v,t)"))


In [41]:
mapa_kb = FolKB(clauses)

In [42]:
mapa_kb.ask(expr("Colorido(ao,tn,q,am,ngs,v,t)"))

{v_21: ao,
 v_22: tn,
 v_24: q,
 v_23: am,
 v_25: ngs,
 v_26: v,
 v_27: t,
 ao: R,
 tn: G,
 am: B,
 q: R,
 ngs: G,
 v: R,
 t: R}

### Preparação para o Exercicio 1:

In [43]:
clauses = []
clauses.append(expr("P(F(x)) ==> P(x)"))
clauses.append(expr("Q(x) ==> P(F(x))"))
clauses.append(expr("P(A)"))
clauses.append(expr("Q(B)"))

In [44]:
exercicio1_kb = FolKB(clauses)

In [45]:
answer = fol_fc_ask(exercicio1_kb, expr("Q(x)")) #testar P(x), Q(A), P(A), P(B)
print(list(answer))

[{x: B}]


In [46]:
clauses = []
clauses.append(expr("P(F(x)) ==> P(x)"))
clauses.append(expr("Q(x) ==> P(F(x))"))
clauses.append(expr("P(A)"))
clauses.append(expr("Q(B)"))

In [47]:
exercicio1_kb = FolKB(clauses)

In [48]:
answer = fol_bc_ask(exercicio1_kb, expr("Q(x)")) #testar P(x), Q(A), P(A), P(B)
print(list(answer))

[{x: B}]


### Respostas

(1) F; Q(A) não é produzido. 

(2) T; via P(F(B)).

(3) T; breadth-first FC é completo para Horn KBs. 

(4) F; loop infinito na primeira regra.

(5) F; P(B) é um exemplo.